In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from math import sqrt
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [ ]:
df=pd.read_csv("../input/datatissue/new_df.csv")

In [ ]:
df.columns


In [ ]:
df_new = df.rename(columns={'Unnamed: 0': 'index'})

In [ ]:
df_new.set_index('index')
df.size

In [ ]:
df

In [ ]:
df_new=df_new.drop('index',1)
df_new = StandardScaler().fit_transform(df_new)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:

pca = PCA(n_components=10)
principalComponents = pca.fit_transform(df_new)

In [ ]:
import matplotlib.pyplot as plt



features = range(pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_, color='blue')
plt.xlabel('PCA caracteristicas')
plt.ylabel('Varianza %')
plt.xticks(features)





In [ ]:
PCA_components = pd.DataFrame(principalComponents)

In [ ]:
plt.scatter(PCA_components[0], PCA_components[1], alpha=.1, color='blue')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')

In [ ]:
pca = PCA(n_components=3)
pc_df = pca.fit_transform(df_new)

In [ ]:
plt.plot(pc_df)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
ks = range(1, 10)
inertias=[]

In [ ]:
for k in ks:
    model = KMeans(n_clusters=k)
    model.fit(pc_df)
    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o', color='blue')
plt.xlabel('# clusters, k')
plt.ylabel('inercia')
plt.xticks(ks)
plt.show()

In [ ]:
dfout = pd.DataFrame(pc_df)
dfout.to_csv('df_pc.csv')

In [ ]:
dfdf=dfout.drop(2,1)

In [ ]:
#pasar todos a positivo
print(min(dfdf[0]),min(dfdf[1]))

for i in range(len(dfdf[0])):
    dfdf[0][i]+=126
for i in range(len(dfdf[1])):
    dfdf[1][i]+=126
print(min(dfdf[0]),min(dfdf[1]))

In [ ]:
import scipy
def dist(el1, el2):
    return scipy.spatial.distance.euclidean(el1, el2)

def generatecenters(k):
    cluster_centers={}
    for i in range (k):
        cluster_centers[str(i)] = list([np.random.randint (0, 172+126),
                                        np.random.randint (0, 172+126)]) 
    return cluster_centers

In [ ]:
centers=generatecenters(3)
centers

In [ ]:
def classify (el, centers):
    minn = 1000000
    for key in centers:
        if dist(el, centers[key]) < minn:
            minn = dist(el, centers[key])
            cluster = key
    return cluster


In [ ]:
def kmeans(df,k,cluster_centers):      
    classes = {}
    for i in range (k):
        classes[str(i)] = []
    for x in range(len(df[0])):
        element = list ([df[0][x],df[1][x]])
        assigned_cluster = classify (element, cluster_centers)
        classes[assigned_cluster].append (element)
    return classes



In [ ]:
a=kmeans(dfdf,3,centers)

In [ ]:


x0=[]
y0=[]
for i in a['0']:
    x0.append(i[0]-126)
    y0.append(i[1]-126)
x1=[]
y1=[]
for i in a['1']:
    x1.append(i[0]-126)
    y1.append(i[1]-126)
x2=[]
y2=[]
for i in a['2']:
    x2.append(i[0]-126)
    y2.append(i[1]-126)

    

    
print(list([np.mean(x0),np.mean(y0)]))
print(list([np.mean(x1),np.mean(y1)]))
print(list([np.mean(x2),np.mean(y2)]))


In [ ]:
plt.scatter(x0,y0,label='cluster 0')
plt.scatter(x1,y1,color='red',label='cluster 1')
plt.scatter(x2,y2,color='green',label='cluster 2')
plt.legend()

In [ ]:
def dbscan(puntos,clusters,radius,tree):
    for i in range (len(puntos)):
        assigned_cluster = -1 
        if clusters[i] == -1:
            assigned_cluster = i 
        else:
            assigned_cluster = clusters[i]
        ind = tree.query_radius([puntos[i]], r=radius, count_only=False, return_distance=False)
    
        is_cluster = False
    
        if len(ind[0]) >= 5:
            is_cluster = True
        if is_cluster:
            for m in ind[0]:
                clusters[m] = assigned_cluster
    return clusters

In [ ]:
def listOfTuples(l1, l2):
    return list(map(lambda x, y:(x,y), l1, l2))

In [ ]:
puntos=listOfTuples(dfdf[0],dfdf[1])

In [ ]:
points=[]
for i in puntos:
    points.append(np.asarray(i))

In [ ]:
from sklearn.neighbors import KDTree

tree = KDTree (points)

In [ ]:
ind = tree.query_radius([points[2]], r=20, count_only=False, return_distance=False)

In [ ]:
len(ind[0])

In [ ]:
radius=10
clusters = [-1 for x in range(len(dfdf))]

In [ ]:
results=dbscan(points,clusters,radius,tree)

In [ ]:
results

In [ ]:
#postprocess
results
fcl={'0':[],
    '1':[],
    '2':[],
    '3':[],
    '4':[],
    }
for i in range(0,len(results)):
    if results[i]==-1:
        fcl['0'].append(points[i])
    if results[i]==8:
        fcl['1'].append(points[i])
    if results[i]==17:
        fcl['2'].append(points[i])
    if results[i]==27:
        fcl['3'].append(points[i])
    if results[i]==164:
        fcl['4'].append(points[i])

In [ ]:
x0=[]
y0=[]
for i in fcl['0']:
    x0.append(i[0]-126)
    y0.append(i[1]-126)
x1=[]
y1=[]
for i in fcl['1']:
    x1.append(i[0]-126)
    y1.append(i[1]-126)
x2=[]
y2=[]
for i in fcl['2']:
    x2.append(i[0]-126)
    y2.append(i[1]-126)
x3=[]
y3=[]
for i in fcl['3']:
    x3.append(i[0]-126)
    y3.append(i[1]-126)
    
x4=[]
y4=[]
for i in fcl['4']:
    x4.append(i[0]-126)
    y4.append(i[1]-126)
    

In [ ]:
plt.scatter(x0,y0,label='Sin cluster')
plt.scatter(x1,y1,color='red',label='cluster 1')
plt.scatter(x2,y2,color='green',label='cluster 2')
plt.scatter(x3,y3,color='yellow',label='cluster 3')
plt.scatter(x4,y4,color='pink',label='cluster 4')
plt.legend()